In [1]:
%pwd

'c:\\Users\\Khalil ABID\\Desktop\\MedicalAgent\\research'

In [2]:
import os
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


c:\Users\Khalil ABID\Desktop\MedicalAgent\.medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Extract text from the PDF file
def load_pdf_files(data):
    loader = DirectoryLoader(
    data,
    glob="*.pdf",
    loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


In [9]:
# Load the PDF file
extracted_data = load_pdf_files("data")

In [11]:
len(extracted_data)

637

In [12]:
#Split the text into smaller chunks
def text_splitter(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks = text_splitter.split_documents(minimal_docs)
    return chunks

In [15]:
chunks = text_splitter(extracted_data)
print(f"Number of chunks: {len(text_splitter(extracted_data))}")

Number of chunks: 5859


In [ ]:
# Embedding the chunks
from langchain.embeddings import HuggingFaceEmbeddings
def download_embeddings():
    """Download the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings


embedding = download_embeddings()

C:\Users\Khalil ABID\AppData\Local\Temp\ipykernel_11052\1976902176.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [17]:
embedding.embed_query("hello world")

[-0.034477293491363525,
 0.031023185700178146,
 0.006734919268637896,
 0.026108955964446068,
 -0.03936200588941574,
 -0.16030244529247284,
 0.06692398339509964,
 -0.00644145580008626,
 -0.047450482845306396,
 0.014758873730897903,
 0.07087531685829163,
 0.05552761256694794,
 0.019193336367607117,
 -0.026251327246427536,
 -0.010109526105225086,
 -0.026940451934933662,
 0.02230745740234852,
 -0.02222668007016182,
 -0.14969263970851898,
 -0.017492998391389847,
 0.007676251698285341,
 0.05435226485133171,
 0.003254401497542858,
 0.031725890934467316,
 -0.08462139964103699,
 -0.029405971989035606,
 0.051595598459243774,
 0.04812406003475189,
 -0.003314854810014367,
 -0.05827920511364937,
 0.04196922481060028,
 0.022210687398910522,
 0.1281888335943222,
 -0.02233893983066082,
 -0.011656275019049644,
 0.06292839348316193,
 -0.032876357436180115,
 -0.0912260189652443,
 -0.03117534890770912,
 0.05269956961274147,
 0.04703487083315849,
 -0.08420306444168091,
 -0.030056191608309746,
 -0.020744830

In [18]:
print("vecto length: ", len(embedding.embed_query("hello world")))

vecto length:  384


In [32]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pinecone_client = Pinecone(pinecone_api_key)

In [22]:
pinecone_client

Creation indexe pinecone

In [24]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pinecone_client.has_index(index_name):
    pinecone_client.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    
index = pinecone_client.Index(index_name)

In [25]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(chunks, embedding, index_name=index_name)


# Add more data to existing vectorstore

In [ ]:
dumydocs = Document(page_content="This is a test document.", metadata={"source": "test"})                    )

In [ ]:
docsearch.add_documents(documents =[dumydocs])

# Creating retriver

In [27]:
retriver = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [28]:
retrived_docs = retriver.get_relevant_documents("What is Acne?")

C:\Users\Khalil ABID\AppData\Local\Temp\ipykernel_11052\45650185.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrived_docs = retriver.get_relevant_documents("What is Acne?")


In [29]:
retrived_docs

[Document(id='1c7cb560-0a33-4622-ad86-52ddeadfa7ca', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='84691d59-65cc-4df8-a8f2-c60ae8446d8b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [33]:
from langchain_groq import ChatGroq

chatModel = ChatGroq(model="openai/gpt-oss-120b")

In [34]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [35]:
system_prompt = (
"You are an Medical assistant for question-answering tasks."
"Use the fllowing pieces of retrived context to ansewwer"
"The question. If you don't know the answer, say you don't know."
"Use three sentences maximum and keep the answer concise and to the point."
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [37]:
question_answer_chain = create_stuff_documents_chain(
    llm=chatModel,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(
    retriver,
    question_answer_chain,
)


In [40]:
response = rag_chain.invoke({"input":"What is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples, especially on the face, chest, and back. It occurs when skin pores become clogged with oil, dead skin cells, and bacteria, leading to inflammation of the sebaceous glands. The most frequent form is acne vulgaris, affecting millions of people.
